In [1]:
import os
import pickle
import json
from transformers import AutoTokenizer
from experiments.pipeline_config import PipelineConfig
from experiments.llm_autointerp.llm_query import perform_llm_autointerp
from typing import Dict

%load_ext autoreload
%autoreload 2

In [18]:
from nnsight import LanguageModel

model = LanguageModel('EleutherAI/pythia-70m-deduped')

First, run autointerp with
```python
cd experiments
python llm_autointerp/run_autointerp_can.py
```

In [2]:
# TODO create run.sh script for above to run llm_query with custom args from this notebook
# For now, manually copy hyperparameters here
repo_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
ae_path = "dictionary_learning/dictionaries/autointerp_test_data/pythia70m_sweep_topk_ctx128_0730/resid_post_layer_3/trainer_2"
ae_path = os.path.abspath(os.path.join(repo_dir, ae_path))

In [21]:
# To save the raw llm output when querying llm, set DEBUG=True
with open(os.path.join(ae_path, "llm_debug_results.pkl"), "r") as f:
    llm_debug_results = pickle.load(f)

with open(os.path.join(ae_path, "node_effects.pkl"), "rb") as f:
    node_effects_classprobe = pickle.load(f)

with open(os.path.join(ae_path, "node_effects_auto_interp.pkl"), "rb") as f:
    node_effects_autointerp = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/share/u/can/sae_eval/dictionary_learning/dictionaries/autointerp_test_data/pythia70m_sweep_topk_ctx128_0730/resid_post_layer_3/trainer_2/llm_debug_results.pkl'

In [4]:
# Show config
cfg = PipelineConfig()
for k, v in cfg.__dict__.items():
    print(f"{k}: {v}")

max_activations_collection_n_inputs: 5000
top_k_inputs_act_collect: 5
probe_train_set_size: 4000
probe_test_set_size: 500
train_set_size: 100
test_set_size: 200
eval_saes_n_inputs: 250
probe_batch_size: 200
probe_epochs: 10
model_dtype: torch.bfloat16
reduced_GPU_memory: False
include_gender: True
use_autointerp: True
force_eval_results_recompute: False
force_max_activations_recompute: False
force_probe_recompute: False
force_node_effects_recompute: False
force_autointerp_recompute: False
dictionaries_path: ../dictionary_learning/dictionaries
probes_dir: trained_bib_probes
attribution_patching_method: attrib
ig_steps: 10
api_llm: claude-3-5-sonnet-20240620
autointerp_api_total_token_per_minute_limit: 400000
autointerp_api_total_requests_per_minute_limit: 4000
num_allowed_tokens_per_minute: 320000
num_allowed_requests_per_minute: 3200
num_tokens_system_prompt: None
prompt_dir: llm_autointerp/
node_effects_attrib_filename: node_effects.pkl
autointerp_filename: node_effects_auto_interp.pk

In [5]:
node_effects_autointerp

{'male / female': tensor([-0., 0., 0.,  ..., -0., -0., 0.]),
 'professor / nurse': tensor([0., 0., 0.,  ..., -0., -0., -0.]),
 'male_professor / female_nurse': tensor([0., 0., 0.,  ..., -0., 0., -0.]),
 'biased_male / biased_female': tensor([0., 0., 0.,  ..., -0., -0., -0.]),
 0: tensor([-0., 0., -0.,  ..., -0., 0., 0.]),
 1: tensor([-0., -0., 0.,  ..., 0., 0., 0.]),
 2: tensor([-0., 0., 0.,  ..., -0., 0., 0.]),
 6: tensor([0., 0., 0.,  ..., -0., 0., 0.]),
 9: tensor([0., 0., -0.,  ..., -0., -0., 0.])}

In [16]:
# Print cfg.num_features_per_class for each class
from experiments.utils_bib_dataset import profession_dict
import torch as t


for cls in cfg.chosen_autointerp_class_names:
    if cls in ['gender', 'professor', 'nurse', ]:
        continue
    else:
        cls_idx = profession_dict[cls]

    top_feature_vals, top_feature_idxs = t.topk(node_effects_classprobe[cls_idx], cfg.num_top_features_per_class)
    autointerp_vals = node_effects_autointerp[cls_idx][top_feature_idxs]
    print(f"{cls}:")
    print(f'feature_idx, probe_effect, autointerp_val')
    for i in range(cfg.num_top_features_per_class):
        print(f'{top_feature_idxs[i]}, {top_feature_vals[i]}, {autointerp_vals[i]}')

accountant:
feature_idx, probe_effect, autointerp_val
1512, 1.140625, 0.0
15917, 0.8515625, 0.0
5376, 0.6875, 0.6875
13927, 0.322265625, 0.322265625
3170, 0.2099609375, 0.0
12957, 0.1767578125, 0.1767578125
12378, 0.1484375, 0.0
9922, 0.10205078125, 0.10205078125
15661, 0.08251953125, 0.08251953125
13033, 0.0810546875, 0.0810546875
6046, 0.078125, 0.0
15482, 0.07177734375, 0.0
10470, 0.050048828125, 0.0
11871, 0.04931640625, 0.0
16363, 0.04541015625, 0.04541015625
5189, 0.04296875, 0.0
9652, 0.041259765625, 0.0
9682, 0.0390625, 0.0
10691, 0.037353515625, 0.0
111, 0.036376953125, 0.036376953125
architect:
feature_idx, probe_effect, autointerp_val
6339, 0.2333984375, 0.0
11224, 0.2265625, 0.2265625
15917, 0.2138671875, 0.0
5376, 0.142578125, 0.0
1512, 0.1396484375, 0.0
1552, 0.12060546875, 0.0
6859, 0.103515625, 0.103515625
9578, 0.091796875, 0.0
8179, 0.08544921875, 0.08544921875
5822, 0.080078125, 0.080078125
6194, 0.0693359375, 0.0
5189, 0.06640625, 0.0
6036, 0.056396484375, 0.0
14889

In [19]:
llm_debug_results.keys()

dict_keys(['16363', '16060'])